In [2]:
from ase import Atoms
from ase.visualize import view
from ase.calculators.emt import EMT
from gpaw import GPAW, FermiDirac
# from ase.optimize import QuasiNewton
from ase.optimize import GPMin

d = 2.5

nanocluster = Atoms('Na6', positions=[(d, 0., 0.), (-d, 0., 0), (0., d, 0.), (0., -d, 0), (0., 0., d), (0., 0., -d)])       # Intuition.
# nanocluster = Atoms('Na6', positions=[(d, 0., 0.), (0, 0., 0), (-d, 0, 0.), (-d/2, d/2, 0), (d/2, d/2, 0.), (0., d, 0.)]) # Testing in-plane (2D).
# view(nanocluster) # For visualization.

nanocluster.set_cell([16, 16, 16]) # 16 x 16 x 16 [Ångström] cell.
nanocluster.center() # The cell must be centered in order to prevent atoms from lying too close to the boundary,
                     # as the boundary conditions are zero by default.

calc = GPAW(nbands=10,
            h=0.25,
            txt='out.txt',
            xc = 'LDA', # 'LDA' is the default.
            occupations=FermiDirac(0.05),
            setups={'Na': '1'},
            mode='lcao',
            basis='dzp')
nanocluster.calc = calc

# The GPMin (Gaussian Process minimizer) produces a model for the Potential Energy Surface using the information about the potential energies
# and the forces of the configurations it has already visited and uses it to speed up BFGS local minimzations.
relax = GPMin(nanocluster, trajectory='relax.traj', logfile='relax.log') # Can also use 'BFGS', 'QuasiNewton', etc.
relax.run(fmax=0.02, steps=100)

E_Na6 = nanocluster.get_potential_energy()

print(f'Final potential energy = {E_Na6} eV.')

calc.write('Na6.gpw', mode = 'all')

Final potential energy = -4.1551131066483675 eV.
